# Compare multiple

In [15]:
import xarray as xr
import hvplot.xarray
from JulesD3D.utils import quickDF, folderPicker, ncFilePicker, multiNcFilePicker
from JulesD3D.processNetCDF import fixMeshGrid
import pandas as pd
# import holoviews as hv
from os import path, walk
import datashader as ds
from IPython.display import Markdown as md
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read first manually because stuff breaks when first is read together with the rest of the files
When merging first .nc file, coords or something are different from following .nc files and xarray doesn't accept it for some reason I don't understand yet

In [16]:
first_ncfiles_folder = '/Users/julesblom/ThesisResults'
first_ncfilename = ncFilePicker(first_ncfiles_folder)
first_ncfilename

Select(description='NetCDF file:', layout=Layout(height='120px', width='100%'), options=('/Users/julesblom/The…

In [141]:
first = xr.open_dataset(first_ncfilename.value)

In [142]:
first.time.size

12

In [143]:
# parent_folder = '/Users/julesblom/ThesisResults/' # open ALL netCDF files in this folder
parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults/'

scenario_results_folder = folderPicker(parent_folder)
scenario_results_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Volumes/Seagate…

In [144]:
# basename = path.basename(scenario_results_folder.value)
scenario_title = path.basename(path.normpath(scenario_results_folder.value)).replace('0.', '0')
md(f"# {scenario_title}")

# Slope085

### 1) Read .nc files all in folder

In [145]:
models = []

for root, dirs, files in walk(scenario_results_folder.value):
    for file in sorted(files):
        if file.endswith('nc'):
            models.append({"path": path.join(scenario_results_folder.value, file), "title": file[9:14]})
pd.DataFrame(models)

,path,title
0,/Volumes/Seagate Expansion Drive/FinalResults/...,Run02
1,/Volumes/Seagate Expansion Drive/FinalResults/...,Run03
2,/Volumes/Seagate Expansion Drive/FinalResults/...,Run04
3,/Volumes/Seagate Expansion Drive/FinalResults/...,Run05
4,/Volumes/Seagate Expansion Drive/FinalResults/...,Run06
5,/Volumes/Seagate Expansion Drive/FinalResults/...,Run07
6,/Volumes/Seagate Expansion Drive/FinalResults/...,Run08
7,/Volumes/Seagate Expansion Drive/FinalResults/...,Run09
8,/Volumes/Seagate Expansion Drive/FinalResults/...,Run10
9,/Volumes/Seagate Expansion Drive/FinalResults/...,Run11


### 2) Or select only some .nc files in folder

In [127]:
# scenario = scenario_results_folder.value[-10:-1]
# result_files = multiNcFilePicker(scenario_results_folder.value)
# result_files

In [128]:
# result_files.value

## Open the DataSets

In [146]:
# array of xarray DataSets
trims = [xr.open_dataset(model_dict['path']) for model_dict in models]

In [147]:
len(trims)

49

In [148]:
channel_index = int(trims[0].M.size/2)

## Sanity check of times, needs to be equal!

In [132]:
times = [trim.time.size for trim in trims]
all_same_time = ((i == trims[0].time.size) for i in times)
all(all_same_time)

True

## Make xarray DataSet with bottom difference per run
Times are all set to first simulations start time to allow comparison in a lineplot

Has to be better way to do this tho

In [57]:
# Ensure that datasets have equal startdates, this is hack for comparing in lineplot with hvplot
same_time = trims[0].time

In [58]:
bottom_diff_per_run_datasets = []
depth_clim = (-0.5, 0.5)

In [59]:
# Make new dataset bottom difference difference PER RUN 
for (i, trim) in enumerate(trims):
    keyword_string = f"{models[i]['title']}"
    print(keyword_string, end=' - ')
    
    # old way dont know why i did it like this: So the DatArrays have the right data var name
    trim[keyword_string] = trim.DPS.isel(time=0) - trim.DPS
    diff_channel_end = trim[keyword_string]

#     diff_channel_end = trim.DPS.isel(time=0) - trim.DPS
#     diff_channel_end.rename(keyword_string)
#     print(diff_channel_end)
    diff_channel_end['time'] = same_time # this is a hack for comparing in lineplot with hvplot
    bottom_diff_per_run_datasets.append(diff_channel_end)

Run02 - Run03 - Run04 - Run05 - Run06 - Run07 - Run08 - Run09 - Run10 - Run11 - Run12 - Run13 - Run14 - Run15 - Run16 - Run17 - Run18 - Run19 - Run20 - Run21 - Run22 - Run23 - Run24 - Run25 - Run26 - Run27 - Run28 - Run29 - Run30 - Run31 - Run32 - Run33 - Run34 - Run35 - Run36 - Run37 - Run38 - Run39 - Run40 - Run41 - Run42 - Run43 - Run44 - Run45 - Run46 - Run47 - Run48 - Run49 - Run50 - 

In [60]:
len(bottom_diff_per_run_datasets)

49

In [61]:
# Make DataSet of list of DataArrays
ds_diff_per_run = xr.merge(bottom_diff_per_run_datasets, compat='override')
ds_diff_per_run = fixMeshGrid(ds_diff_per_run, mystery_flag=True)
ds_diff_per_run

● Fixing mesh grid, assuming a uniform grid 
132 x 182 grid


<xarray.Dataset>
Dimensions:  (M: 132, N: 182, time: 12)
Coordinates:
    XZ       (M, N) int64 0 0 0 0 0 0 0 ... 26100 26100 26100 26100 26100 26100
    YZ       (M, N) int64 0 200 400 600 800 ... 35400 35600 35800 36000 36200
  * time     (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
Dimensions without coordinates: M, N
Data variables:
    Run02    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run03    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run04    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run05    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run06    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run07    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run08    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run09    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run10    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run11    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run12    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run13    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run14    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run15    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run16    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run17    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run18    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run19    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run20    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run21    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run22    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run23    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run24    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run25    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run26    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run27    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run28    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run29    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run30    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run31    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run32    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run33    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run34    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run35    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run36    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run37    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run38    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run39    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run40    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run41    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run42    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run43    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run44    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run45    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run46    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run47    (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run48    (M, N, time) float32 0.0 0.0 0.0 0.0 0.

## Get max & mean thickness per run

In [28]:
threshold_m = 0.002
f"Threshold is {threshold_m * 100} cm"

'Threshold is 0.2 cm'

In [29]:
# there's also a prop of gridcell area in trim DataSet: GSQS
# but square gridcells make this easy
grid_cell_area_m2 = 200 * 200
grid_cell_area_km2 = grid_cell_area_m2 / (1000 * 1000)
grid_cell_area_km2

0.04

In [ ]:
thickness_dict ={'name': [], 'mean': [], 'max': [], 'area': [], 'x_max': [], 'y_max': [], 'x_centroid': [], 'y_centroid': []}
for var in ds_diff_per_run.data_vars:
    print(var, end=' - ')
    basin_part = ds_diff_per_run[var].isel(N=slice(75,None)) #.thin({'time': 12 })
    threshold_basin = basin_part.where(basin_part > threshold_m)

    mean_deposit = threshold_basin.isel(time=-1).mean().values
    max_deposit = threshold_basin.max().values

    # count number of gridcells
    area_deposit = threshold_basin.isel(time=-1).count().values * grid_cell_area_km2
    
    x_coord_depocenter = basin_part.where(basin_part == max_deposit, drop=True).squeeze().XZ.values
    y_coord_depocenter = basin_part.where(basin_part == max_deposit, drop=True).squeeze().YZ.values

    x_coord_centroid = float(basin_part.XZ.where(basin_part > threshold_m).mean().values)
    y_coord_centroid = float(basin_part.YZ.where(basin_part > threshold_m).mean().values)
    
    # not pretty but it works
    thickness_dict['area'].append(area_deposit)
    thickness_dict['name'].append(var[3:None])
    thickness_dict['mean'].append(mean_deposit * 100) 
    thickness_dict['max'].append(max_deposit * 100) 
    thickness_dict['x_max'].append(x_coord_depocenter)
    thickness_dict['y_max'].append(y_coord_depocenter)
    thickness_dict['x_centroid'].append(x_coord_centroid)
    thickness_dict['y_centroid'].append(y_coord_centroid)    

In [ ]:
thicknesses_per_run = pd.DataFrame(thickness_dict)
thicknesses_per_run.to_csv(f'csv/{scenario_title}_thicknesses_per_run.csv')

## Line profile sed/ero per run 

In [111]:
cross_section_channel =  ds_diff_per_run.isel(M=channel_index, time=-1)
cross_section_channel.hvplot.line(ylim=depth_clim, ylabel='') # groupby='time',

:NdOverlay   [Variable]
   :Curve   [YZ]   (value)

In [112]:
# export lineplot values
# kk_csv = {}

# kk_csv['y'] = cross_section_channel.YZ.values

# for datavar in cross_section_channel:
#     kk_csv[datavar] = cross_section_channel[datavar].values
    
# kk_csv

# Total accumulated sedimentation/erosion

In [133]:
bottom_diff_datasets_cumu = []

# Accumulated bottom difference 
for (i, trim) in enumerate(trims):
    keyword_string = f"{models[i]['title']}_diff"
    trim[keyword_string] = first.DPS.isel(time=0) - trim.DPS
    
    diff_channel_end = trim[keyword_string]
    diff_channel_end['time'] = same_time # this is hack for comparing in lineplot with hvplot
    bottom_diff_datasets_cumu.append(diff_channel_end)

In [134]:
ds_bottom_diff_cumu = xr.merge(bottom_diff_datasets_cumu, compat='override')
ds_bottom_diff_cumu

<xarray.Dataset>
Dimensions:     (M: 132, N: 182, time: 12)
Coordinates:
    XZ          (M, N) float64 ...
    YZ          (M, N) float64 ...
  * time        (time) datetime64[ns] 2020-02-01T20:30:00 ... 2020-02-02T16:18:00
Dimensions without coordinates: M, N
Data variables:
    Run02_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run03_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run04_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run05_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run06_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run07_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run08_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run09_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run10_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run11_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run12_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run13_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run14_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run15_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run16_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run17_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run18_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run19_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run20_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run21_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run22_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run23_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run24_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run25_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run26_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run27_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run28_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run29_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run30_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run31_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run32_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run33_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run34_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run35_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run36_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run37_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run38_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run39_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run40_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run41_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run42_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run43_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run44_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run45_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run46_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    Run47_diff  (M, N, time) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 

In [135]:
# fix delft3d shitty output meshgrid with missing values due to enclosure
ds_bottom_diff_total = fixMeshGrid(ds_bottom_diff_cumu, mystery_flag=True)

● Fixing mesh grid, assuming a uniform grid 
132 x 182 grid


In [136]:
thickness_dict_total = {'name': [], 'mean': [], 'max': [], 'x': [], 'y': []}
for var in ds_bottom_diff_total.data_vars:
    basin_part = ds_bottom_diff_total[var].isel(N=slice(75,None)) #.thin({'time': 12 })
    threshold_basin = basin_part.where(basin_part > threshold_m)

    mean_deposit = threshold_basin.isel(time=-1).mean().values
    max_deposit = threshold_basin.max().values
    
    x_coord_depocenter = basin_part.where(basin_part == max_deposit, drop=True).squeeze().XZ.values
    y_coord_depocenter = basin_part.where(basin_part == max_deposit, drop=True).squeeze().YZ.values

    # areas & centroid?
    
    # = ['Mean thickness', 'Max thickness', 'Max thick X', 'Max thick Y']
    thickness_dict_total['name'].append(var)
    thickness_dict_total['mean'].append(mean_deposit * 100) 
    thickness_dict_total['max'].append(max_deposit * 100) 
    thickness_dict_total['x'].append(x_coord_depocenter)
    thickness_dict_total['y'].append(y_coord_depocenter)

NameError: name 'threshold_m' is not defined

In [ ]:
pd.DataFrame(thickness_dict_total).to_csv(f'csv/{scenario_title}_thicknesses_cumulative.csv')

In [ ]:
# ds_bottom_diff_cumu_end = ds_bottom_diff_cumu.thin({'time': 12})
# ds_bottom_diff_cumu_end.time

In [ ]:
# map_clim = (-0.3, 0.3)

In [ ]:
# quadmesh_plotting_options = dict(cmap='PiYG', height=550, width=400, clabel='m', clim=map_clim)

# all_end_plots = [ds_bottom_diff_cumu_end[model_name].hvplot.quadmesh('XZ', 'YZ', **quadmesh_plotting_options, title=model_name) for model_name in ds_bottom_diff_cumu_end.data_vars]

In [ ]:
# all_end_plots[0] + all_end_plots[1] + all_end_plots[2]

In [ ]:
# all_end_plots[3] + all_end_plots[4] + all_end_plots[6]

In [41]:
# all_end_plots[7] 

### Now what if I write a for loop that uses hv.save() to writes these maps to an imagefile

# Cumulative sedimentation erosion profile

In [149]:
ds_bottom_diff_total.isel(M=channel_index, time=-1).hvplot.line(
    ylim=(-6.5,6.5),
    legend=False,
    ylabel='Sedimentation/Erosion (m)',
    xlabel="Length in y-direction (m)",
    title=f"{scenario_title} Cumulative sedimentation/erosion profile through channel (X=13 200)",
)

:NdOverlay   [Variable]
   :Curve   [YZ]   (value)

In [150]:
ds_bottom_diff_total.isel(M=channel_index, N=slice(13,75), time=-1).hvplot.line(
    ylim=(-6.5,6.5),
    legend=False,
    ylabel='Sedimentation/Erosion (m)',
    xlabel="Length in y-direction (m)",    
    title=f"{scenario_title} Cumulative sedimentation/erosion profile through channel (X=13 200)",
)

:NdOverlay   [Variable]
   :Curve   [YZ]   (value)